# <u>TP2</u>

## par Antoine Boissinot  - -  François Gaudreault  - -  Béatrice Lessard-Hamel

## Présenté à M. Philippe Després

### Date de remise : 5 mars 2021

## Numéro 1  

## Numéro 2 

## Numéro 3  

## Numéro 4  

## Numéro 5  

## Numéro 6  

## Numéro 7  

## Numéro 8  

## Numéro 9  